# TP : Classification de critiques IMDb avec PyTorch

## Objectifs

- **Manipuler le dataset IMDb** en utilisant la librairie [datasets](https://huggingface.co/docs/datasets).
- **Prétraiter le texte** : tokenisation, construction d’un vocabulaire, conversion des textes en séquences d’indices et mise à la même longueur (padding).
- **Construire un modèle simple** en PyTorch composé d’une couche d’**embedding** et d’une couche dense.
- **Entraîner et évaluer le modèle** sur un problème de classification binaire (critique positive / critique négative).

## Contexte

Nous souhaitons construire un modèle de classification de critiques de films. Pour cela, nous allons :
- Charger et découper le dataset IMDb.
- Préparer les données pour les faire passer dans un modèle PyTorch.
- Construire un réseau de neurones simple qui se compose d’une couche d’embedding (pour transformer chaque mot en vecteur) et d’une couche dense (pour réaliser la classification).
- Entraîner le modèle et évaluer ses performances.

---

## Points de discussion en fin de TP

- **Simplicité du modèle :** Ce modèle ne prend en compte que l’information globale par une moyenne des embeddings. Quelles seraient les limites de cette approche ?
- **Prétraitement du texte :** Quels outils (par exemple, spaCy, nltk, Hugging Face Tokenizers) pourraient être utilisés pour améliorer la tokenisation et la gestion du vocabulaire ?
- **Améliorations possibles :** Proposez des idées afin d'améliorez ce modèle